In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,32630
2,Huelva,Confirmados PDIA 14 días,1349
3,Huelva,Tasa PDIA 14 días,"262,87585010815127"
4,Huelva,Confirmados PDIA 7 días,581
5,Huelva,Total Confirmados,32812
6,Huelva,Curados,30409
7,Huelva,Fallecidos,388


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  32630.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  8883.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 571 personas en los últimos 7 días 

Un positivo PCR cada 247 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,32630.0,581.0,1349.0,513170.0,113.217842,262.875850,102.0
Huelva-Costa,18804.0,355.0,799.0,289548.0,122.604888,275.947339,47.0
Condado-Campiña,10590.0,173.0,426.0,156231.0,110.733465,272.673157,44.0
Huelva (capital),8883.0,252.0,583.0,143837.0,175.198315,405.319911,35.0
Almonte,1878.0,24.0,62.0,24507.0,97.931203,252.988942,12.0
Sierra de Huelva-Andévalo Central,2885.0,52.0,120.0,67391.0,77.161639,178.065320,11.0
Bollullos Par del Condado,1022.0,17.0,46.0,14387.0,118.162230,319.733092,7.0
Isla Cristina,2170.0,17.0,22.0,21393.0,79.465246,102.837377,6.0
Valverde del Camino,635.0,24.0,60.0,12750.0,188.235294,470.588235,6.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cortelazor,13.0,4.0,5.0,299.0,1337.792642,1672.240803,0.0
San Juan del Puerto,557.0,25.0,57.0,9411.0,265.646584,605.674211,5.0
Puebla de Guzmán,134.0,11.0,18.0,3092.0,355.756792,582.147477,2.0
Valverde del Camino,635.0,24.0,60.0,12750.0,188.235294,470.588235,6.0
Palos de la Frontera,700.0,13.0,50.0,11742.0,110.713677,425.821836,1.0
Huelva (capital),8883.0,252.0,583.0,143837.0,175.198315,405.319911,35.0
Villarrasa,180.0,1.0,8.0,2211.0,45.228403,361.827227,0.0
Almendro (El),39.0,3.0,3.0,840.0,357.142857,357.142857,0.0
Moguer,1357.0,38.0,73.0,21867.0,173.777839,333.836374,5.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Villarrasa,180.0,1.0,8.0,2211.0,45.228403,361.827227,0.0,0.125000
Minas de Riotinto,128.0,1.0,7.0,3812.0,26.232949,183.630640,0.0,0.142857
Manzanilla,103.0,1.0,5.0,2118.0,47.214353,236.071766,0.0,0.200000
Punta Umbría,840.0,8.0,32.0,15355.0,52.100293,208.401172,0.0,0.250000
Palos de la Frontera,700.0,13.0,50.0,11742.0,110.713677,425.821836,1.0,0.260000
Bonares,291.0,6.0,20.0,6060.0,99.009901,330.033003,3.0,0.300000
Trigueros,327.0,7.0,22.0,7862.0,89.035869,279.827016,2.0,0.318182
Lepe,2151.0,8.0,24.0,27880.0,28.694405,86.083214,2.0,0.333333
Bollullos Par del Condado,1022.0,17.0,46.0,14387.0,118.162230,319.733092,7.0,0.369565
